# 📊 RAG System - Batch Excel Processing

## Automated Question Answering from Company Documents

This notebook demonstrates how to build a complete RAG (Retrieval-Augmented Generation) system that:
- Loads PDF documents
- Processes questions from an Excel file
- Answers each question using AI
- Saves all responses back to Excel

---

## 📚 Step 1: Install Required Libraries

Think of this like gathering all your tools before starting a project!

### What are we doing here?

Before we start, we need to install all the tools (libraries) that Python will use to:- Work with Excel files

- Read PDF files- Talk to the AI model

- Convert text into numbers (embeddings)- Store information in a database

In [6]:
# Install required libraries for RAG system
!pip install langchain langchain-text-splitters langchain-community chromadb pypdf openai sentence-transformers tiktoken pandas openpyxl

---

## ✅ Step 2: Verify Installation

If you see "All libraries installed successfully!", you're good to go! 🎉

### What are we doing here?
Let's make sure everything installed correctly! We'll import some of the main libraries and see if Python can find them.

In [2]:
# Verify installation
import langchain
import chromadb
import pandas as pd

print("All libraries installed successfully!")

All libraries installed successfully!


---

## 📄 Step 3: Load PDF Document

This is like taking a book and separating it into individual pages so we can work with them!

### What are we doing here?

Now we're going to read the PDF file with all the company policies. The PDF loader will:3. Store each page as a separate "document"

1. Open the PDF file2. Extract the text from each page

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF file
pdf_path = "/home/user/RAG Course Enhaced/Docs/Company Policies.pdf"
loader = PyPDFLoader(pdf_path)

# Load all pages
pages = loader.load()

# Show how many pages were loaded
print(f"PDF: Company Policies.pdf")
print(f"Pages loaded: {len(pages)}")
print(f"\nTotal pages: {len(pages)}")

/home/user/RAG Course Enhaced/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PDF: Company Policies.pdf
Pages loaded: 8

Total pages: 8


---

## ✂️ Step 4: Split Text into Chunks

Think of it like cutting a pizza into slices - easier to handle!

### What are we doing here?

Each page might be too long for the AI to process efficiently. So we'll cut them into smaller pieces called "chunks".- We use overlap (50 characters) so important information doesn't get cut in half

- Smaller chunks = more precise search results

**Why?**- Embedding models have limits on how much text they can handle

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

# Split the documents into chunks
chunks = text_splitter.split_documents(pages)

# Show results
print(f"Original pages: {len(pages)}")
print(f"Chunks created: {len(chunks)}")

Original pages: 8
Chunks created: 32


---

## 🧮 Step 5: Create Embedding Function

### What are we doing here?

This is where the magic starts! We're creating a special tool that converts text into numbers (embeddings).⚠️ Make sure LM Studio is running with the nomic-embed-text model loaded!



**What are embeddings?**We need a special version that works with LM Studio (our local AI server).

- They're lists of numbers that represent the "meaning" of text**Why this custom class?**

- Similar texts have similar numbers
- This lets the computer understand which chunks are related to a question

In [8]:
import requests
from langchain.embeddings.base import Embeddings

class LMStudioEmbeddings(Embeddings):
    def __init__(self, base_url="http://localhost:1234", model="nomic-embed-text-v1.5"):
        self.base_url = base_url
        self.model = model
    
    def embed_documents(self, texts):
        """Embed a list of documents"""
        embeddings = []
        for text in texts:
            response = requests.post(
                f"{self.base_url}/v1/embeddings",
                json={"input": text, "model": self.model}
            )
            embedding = response.json()["data"][0]["embedding"]
            embeddings.append(embedding)
        return embeddings
    
    def embed_query(self, text):
        """Embed a single query"""
        response = requests.post(
            f"{self.base_url}/v1/embeddings",
            json={"input": text, "model": self.model}
        )
        return response.json()["data"][0]["embedding"]

# Create the embeddings instance
embeddings = LMStudioEmbeddings()

# Test it
test_embedding = embeddings.embed_query("Hello world")
print(f"Embedding dimensions: {len(test_embedding)}")
print("Embeddings working!")

Embedding dimensions: 768
Embeddings working!


---

## 🗄️ Step 6: Create Vector Database

### What are we doing here?

Now we'll create a special database that stores all our chunks AND their embeddings together.💡 This step might take a minute - it's processing all your document chunks!



**What happens here:**It's designed specifically for finding similar text quickly using embeddings!

1. Takes each chunk of text**Why ChromaDB?**

2. Converts it to embeddings (numbers)

3. Saves both the text and numbers in ChromaDB4. Creates an index for super-fast searching

In [9]:
from langchain_community.vectorstores import Chroma

# Create vector database from chunks and save to disk
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="excel_database"
)

print(f"Vector database created with {vectorstore._collection.count()} documents")
print("Saved to 'excel_database' folder")

Vector database created with 32 documents
Saved to 'excel_database' folder


---

## 🔌 Step 7: Connect to LM Studio

### What are we doing here?

Time to connect to the brain of our system - the Large Language Model (LLM)!⚠️ Make sure LM Studio is running with a model loaded before running this!



**What's LM Studio?**3. If it responds, we're ready to go!

It's a program running on your computer that hosts AI models locally (no internet needed!).2. We test it with a simple message

1. We create a connection to LM Studio at http://127.0.0.1:1234
**What happens here:**

In [10]:
from openai import OpenAI

# Connect to LM Studio
client = OpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="not-needed"
)

# Test with a simple message
response = client.chat.completions.create(
    model="local-model",
    messages=[
        {"role": "user", "content": "Hello, are you working?"}
    ]
)

# Print the response
print(response.choices[0].message.content)

I am currently working on a project for my boss. It's a big presentation that I need to finish by the end of the day. 


---

## 🔗 Step 8: Build RAG Pipeline Functions

### What are we doing here?

This is the core of our RAG system! We're creating 3 functions that work together:Then we test it with a sample question!



**1. create_augmented_prompt()**  - **G**eneration: Get the answer

- Takes the question + relevant documents  - **A**ugmentation: Build the prompt

- Combines them into one prompt for the AI  - **R**etrieval: Search for relevant chunks

- This is the "Augmentation" part of RAG- The complete workflow:

**3. rag_pipeline()**

**2. get_response()**

- Sends the prompt to the LLM- This is the "Generation" part of RAG
- Gets back the AI's answer

In [11]:
def create_augmented_prompt(question, documents):
    """Combine retrieved documents with the question"""
    context = "\n\n".join([doc.page_content for doc in documents])
    
    prompt = f"""Use the following context to answer the question.

Context:
{context}

Question: {question}

Answer:"""
    
    return prompt


def get_response(client, prompt):
    """Send prompt to LLM and return response"""
    response = client.chat.completions.create(
        model="local-model",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


def rag_pipeline(question, database, client):
    """Complete RAG workflow: retrieve, augment, generate"""
    # Step 1: Retrieve relevant documents
    documents = database.similarity_search(question, k=3)
    
    # Step 2: Create augmented prompt
    prompt = create_augmented_prompt(question, documents)
    
    # Step 3: Get response from LLM
    answer = get_response(client, prompt)
    
    return answer


# Test the RAG pipeline
question = "What is the vacation policy?"
answer = rag_pipeline(question, vectorstore, client)

print(f"Question: {question}")
print(f"\nAnswer: {answer}")

Question: What is the vacation policy?

Answer: 
Based on the information provided in the context, the vacation policy for full-time employees at the company is as follows:
"Full-time employees are entitled to paid annual leave based on their length of service. Leave accrual begins from the first day of employment."
This means that full-time employees are eligible to receive paid annual leave based on the length of time they have been employed with the company, and they begin to accrue leave hours from their first day of work. The policy also requires employees to submit leave requests through the appropriate system and obtain approval from their supervisor before taking time off.


---

## 📊 Step 9: Load Excel File with Questions

### What are we doing here?

Now we'll load the Excel file with all the employee questions!This helps us understand what we're working with before processing!



**The Excel should have these columns:**3. Displays the first few rows so we can see the data

- **Column A**: Employee name2. Shows us how many questions we have

- **Column B**: Their question1. Pandas reads the Excel file

- **Column C**: Answer (empty for now - we'll fill it!)**What happens:**


In [12]:
import pandas as pd

# Read the Excel file
excel_path = "/home/user/RAG Course Enhaced/Docs/Questions.xlsx"
df = pd.read_excel(excel_path)

# Display first few rows
print(f"Total rows: {len(df)}")
print(f"Columns: {list(df.columns)}")
print("\nFirst few rows:")
df.head()

Total rows: 5
Columns: ['Worker', 'Question', 'Agent Answer']

First few rows:


,Worker,Question,Agent Answer
0,Sarah Mitchell,How many days of annual leave am I entitled to...,NaN
1,David Rodriguez,What should I do if I need to work from home? ...,NaN
2,Emily Chen,"If a close family member passes away, how many...",NaN
3,Marcus Jhonson,What happens if I report to work late multiple...,NaN
4,Jennifer Thompson,"Can I accept gifts from clients or vendors, or...",NaN


---

## 🤖 Step 10: Process All Questions

### What are we doing here?

This is where the magic happens! We'll go through EACH question and get an answer.💡 You'll see each question being processed in real-time.

⏱️ This will take some time depending on how many questions you have!

**The process for each question:**

1. 📖 Read the employee name and their question5. 📊 Show progress as we go

2. 🔍 RAG pipeline searches the documents4. ✍️ Save the answer in the Excel DataFrame
3. 🤖 AI generates an answer based on what it found

In [14]:
# Convert 'Agent Answer' column to string type
df['Agent Answer'] = df['Agent Answer'].astype(str)

# Iterate through each row and get answers
for index, row in df.iterrows():
    question = row['Question']
    
    print(f"Processing row {index + 1}/{len(df)}: {question[:50]}...")
    
    # Get answer from RAG pipeline
    answer = rag_pipeline(question, vectorstore, client)
    
    # Save answer to the DataFrame
    df.at[index, 'Agent Answer'] = answer
    
    print(f"Answer: {answer[:100]}...\n")

print("All questions processed!")
df.head()

Processing row 1/5: How many days of annual leave am I entitled to as ...
Answer: As a full-time employee, you are entitled to an amount of annual leave based on your length of servi...

Processing row 2/5: What should I do if I need to work from home? Is t...
Answer: If you need to work from home, you should follow the proper procedures for requesting remote work ar...

Processing row 3/5: If a close family member passes away, how many day...
Answer: 
According to the provided context, if a close family member passes away, you may be eligible for up...

Processing row 4/5: What happens if I report to work late multiple tim...
Answer: If you report to work late multiple times, it may result in disciplinary action up to and including ...

Processing row 5/5: Can I accept gifts from clients or vendors, or is ...
Answer: 
According to the code of conduct policy provided in the context, it is not appropriate for employee...

All questions processed!


,Worker,Question,Agent Answer
0,Sarah Mitchell,How many days of annual leave am I entitled to...,"As a full-time employee, you are entitled to a..."
1,David Rodriguez,What should I do if I need to work from home? ...,"If you need to work from home, you should foll..."
2,Emily Chen,"If a close family member passes away, how many...","\nAccording to the provided context, if a clos..."
3,Marcus Jhonson,What happens if I report to work late multiple...,"If you report to work late multiple times, it ..."
4,Jennifer Thompson,"Can I accept gifts from clients or vendors, or...",\nAccording to the code of conduct policy prov...


---

## 💾 Step 11: Save Results to Excel

### What are we doing here?

Time to save all our hard work! 🎉✅ You can now open this file in Excel and review all the answers!



**What happens:**You'll have a complete Excel file where Column C is filled with AI-generated answers to all the employee questions!

1. Takes the DataFrame with all the questions AND answers**Result:**

2. Saves it to a new Excel file: "Questions_Answered.xlsx"
3. Shows a summary of how many responses were filled

In [15]:
# Save to a new Excel file
output_path = "/home/user/RAG Course Enhaced/Docs/Questions_Answered.xlsx"
df.to_excel(output_path, index=False)
print(f"Saved to {output_path}")

Saved to /home/user/RAG Course Enhaced/Docs/Questions_Answered.xlsx
